In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)


from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *


2023-12-31 14:56:06.560917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 14:56:06.561022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 14:56:06.658990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 14:56:06.825242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 14:56:07.809187: W tensorflow/compiler/tf2

In [3]:
faces = np.load("olivetti_faces.npy")
faces_target = np.load("olivetti_faces_target.npy")

target_df = pd.DataFrame(faces_target)

faces_reshape = faces.reshape((faces.shape[0],faces.shape[1]*faces.shape[2]))
faces_df = pd.DataFrame(faces_reshape)
faces_df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.309917,0.367769,0.417355,0.442149,0.528926,0.607438,0.657025,0.677686,0.690083,0.685950,...,0.665289,0.669421,0.652893,0.661157,0.475207,0.132231,0.148760,0.152893,0.161157,0.157025
1,0.454545,0.471074,0.512397,0.557851,0.595041,0.640496,0.681818,0.702479,0.710744,0.702479,...,0.136364,0.157025,0.136364,0.148760,0.152893,0.152893,0.152893,0.152893,0.152893,0.152893
2,0.318182,0.400826,0.491736,0.528926,0.586777,0.657025,0.681818,0.685950,0.702479,0.698347,...,0.074380,0.132231,0.181818,0.136364,0.128099,0.148760,0.144628,0.140496,0.148760,0.152893
3,0.198347,0.194215,0.194215,0.194215,0.190083,0.190083,0.243802,0.404959,0.483471,0.516529,...,0.652893,0.636364,0.657025,0.685950,0.727273,0.743802,0.764463,0.752066,0.752066,0.739669
4,0.500000,0.545455,0.582645,0.623967,0.648760,0.690083,0.694215,0.714876,0.723140,0.731405,...,0.190083,0.161157,0.177686,0.173554,0.177686,0.177686,0.177686,0.177686,0.173554,0.173554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.400826,0.495868,0.570248,0.632231,0.648760,0.640496,0.661157,0.636364,0.665289,0.698347,...,0.388430,0.396694,0.264463,0.099174,0.181818,0.243802,0.247934,0.161157,0.157025,0.136364
396,0.367769,0.367769,0.351240,0.301653,0.247934,0.247934,0.367769,0.512397,0.574380,0.628099,...,0.380165,0.334711,0.289256,0.285124,0.338843,0.404959,0.458678,0.487603,0.512397,0.549587
397,0.500000,0.533058,0.607438,0.628099,0.657025,0.632231,0.657025,0.669421,0.673554,0.702479,...,0.194215,0.148760,0.152893,0.161157,0.161157,0.173554,0.157025,0.177686,0.148760,0.190083
398,0.214876,0.219008,0.219008,0.223140,0.210744,0.202479,0.276859,0.400826,0.487603,0.549587,...,0.446281,0.392562,0.367769,0.409091,0.479339,0.524793,0.545455,0.574380,0.590909,0.603306


In [4]:
pd.DataFrame(np.array(faces_df.iloc[0,:]).reshape((64,64)))

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.309917,0.367769,0.417355,0.442149,0.528926,0.607438,0.657025,0.677686,0.690083,0.685950,...,0.698347,0.677686,0.657025,0.632231,0.566116,0.524793,0.446281,0.371901,0.330579,0.305785
1,0.342975,0.404959,0.438017,0.471074,0.553719,0.623967,0.669421,0.685950,0.690083,0.685950,...,0.698347,0.673554,0.657025,0.636364,0.603306,0.512397,0.442149,0.371901,0.338843,0.314050
2,0.342975,0.417355,0.450413,0.512397,0.574380,0.644628,0.677686,0.690083,0.694215,0.694215,...,0.685950,0.677686,0.657025,0.644628,0.603306,0.570248,0.433884,0.380165,0.338843,0.297521
3,0.305785,0.409091,0.471074,0.524793,0.595041,0.652893,0.673554,0.677686,0.685950,0.685950,...,0.690083,0.677686,0.657025,0.648760,0.623967,0.541322,0.471074,0.384298,0.347107,0.264463
4,0.334711,0.421488,0.475207,0.582645,0.640496,0.665289,0.665289,0.673554,0.681818,0.677686,...,0.702479,0.685950,0.661157,0.644628,0.644628,0.553719,0.487603,0.388430,0.367769,0.334711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.206612,0.198347,0.194215,0.206612,0.198347,0.198347,0.185950,0.516529,0.677686,0.657025,...,0.611570,0.611570,0.615702,0.640496,0.512397,0.177686,0.144628,0.157025,0.152893,0.161157
60,0.206612,0.202479,0.198347,0.214876,0.190083,0.202479,0.173554,0.528926,0.681818,0.677686,...,0.623967,0.632231,0.628099,0.657025,0.500000,0.161157,0.157025,0.161157,0.165289,0.169421
61,0.214876,0.206612,0.223140,0.202479,0.214876,0.157025,0.181818,0.545455,0.685950,0.702479,...,0.628099,0.648760,0.648760,0.661157,0.495868,0.148760,0.157025,0.152893,0.165289,0.173554
62,0.202479,0.210744,0.210744,0.206612,0.202479,0.107438,0.190083,0.566116,0.685950,0.702479,...,0.644628,0.665289,0.648760,0.661157,0.487603,0.136364,0.152893,0.148760,0.161157,0.165289


In [5]:
X, X_test, y, y_test=train_test_split(faces_df.values, faces_target, test_size=0.2, stratify=faces_target,shuffle=True, random_state=42)
print("X_train shape:",X.shape)
print("y_train shape:{}".format(y.shape))

X_train shape: (320, 4096)
y_train shape:(320,)


In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score



In [7]:
def lda_model (X,y, lda_object):
    num_folds = 4

    # Create a stratified k-fold cross-validation object
    stratified_kfold = StratifiedKFold(n_splits=num_folds,random_state=42,shuffle=True)
    lda = lda_object
    accuracy1 = 0
    accuracy2 = 0
    initial_theta = None
    # Perform stratified k-fold cross-validation
    for train_index, test_index in stratified_kfold.split(X, y):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        lda = lda_object

        #if initial_theta is not None:
            #lda.coef_ = initial_theta
        # Fit the model on the training data
        history = lda.fit(X_train, y_train)

        initial_theta = lda.coef_

        y_pred = lda.predict(X_val)

        # Calculate accuracy
        accuracy1 += accuracy_score(y_val, y_pred)
        
        y_pred = lda.predict(X_train)
        accuracy2 += accuracy_score(y_train, y_pred)
    print(f"Accuracy val: {accuracy1/4:.2f}")
    print(f"Accuracy train: {accuracy2/4:.2f}")
    return lda
    



In [8]:
lda_object = LinearDiscriminantAnalysis()
lda = lda_model(X,y,lda_object)


lda.fit(X, y)
y_pred = lda.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy test: {accuracy:.2f}")
lda = LinearDiscriminantAnalysis()

Accuracy val: 0.96
Accuracy train: 1.00
Accuracy test: 1.00


In [9]:
from sklearn.model_selection import GridSearchCV
#lda = LinearDiscriminantAnalysis(solver='svd', shrinkage=None, n_components=None, store_covariance=False)
param_grid = {
    'n_components': [1, 2, 3, 4, 5, 10, None], }
grid_search = GridSearchCV(lda, param_grid, cv=4,scoring="accuracy",verbose=1,n_jobs=-1)  # You can adjust the number of folds (cv) as needed

# Perform the grid search on your data
grid_search.fit(X, y)
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

Fitting 4 folds for each of 7 candidates, totalling 28 fits
Best Parameters:  {'n_components': 1}
Best Accuracy:  0.96875


In [10]:
lda_object = LinearDiscriminantAnalysis(shrinkage= 'auto', solver = 'lsqr',n_components =1)
lda = lda_model(X,y,lda_object)


lda.fit(X, y)
y_pred = lda.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy test: {accuracy:.2f}")
lda = LinearDiscriminantAnalysis()

KeyboardInterrupt: 

In [11]:
X_train, X_test, y_train, y_test=train_test_split(faces_df.values, faces_target, test_size=0.2, stratify=faces_target,shuffle=True, random_state=42)
X_lda_train = lda_object.fit_transform(X_train, y_train)
X_lda_test = lda_object.transform(X_test)


from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

# Assuming you have X_train, X_test, y_train, and y_test

# Select only 10 people for visualization
selected_people = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
selected_indices_train = [i for i, label in enumerate(y_train) if label in selected_people]

# Filter data for the selected people
X_selected_train = X_train[selected_indices_train]
y_selected_train = y_train[selected_indices_train]

# Fit LDA model for dimensionality reduction
lda_object = LinearDiscriminantAnalysis(n_components=2)
X_lda_train = lda_object.fit_transform(X_selected_train, y_selected_train)
plt.figure(figsize=(10,10))
# Plot the 2D scatter plot for the selected people in the training data
for class_label in set(y_selected_train):
    plt.scatter(X_lda_train[y_selected_train == class_label, 0],
                X_lda_train[y_selected_train == class_label, 1],
                label=f'Person {class_label}')

plt.title('2D Scatter Plot of Selected Training Data with LDA Dimensionality Reduction')
plt.xlabel('LDA Component 1')
plt.ylabel('LDA Component 2')
plt.legend()
plt.show()



In [ ]:

cm = confusion_matrix(y_test, y_pred)

plt.figure(1, figsize=(12, 8))
sns.heatmap(cm, fmt='g')  # annot=True to display the counts, fmt='g' to format as integers

plt.xlabel('Predicted')
plt.ylabel('True Value')
plt.title('Confusion Matrix')

plt

In [ ]:
# Apply LDA transformation to the training and testing data
lda_object = LinearDiscriminantAnalysis(n_components = 9)
lda = lda_model(X_pca_train,y_pca_train,lda_object)

X_train_lda = lda.transform(X_pca_train)
X_test_lda = lda.transform(X_pca_test)